# Multiselect map in JS
from memory i guess SO DRAMATIC lol

In [1]:
import numpy as np
import pandas as pd

import bokeh.io

import panel as pn
import datetime as dt

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)

bokeh.io.output_notebook()

pn.extension()

Loading BokehJS ...

# Load and prep data

In [2]:
# Helper fxn to set coords right
def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

In [3]:
# Load in the metadata
ferm_locations = pd.read_csv("./data/ferm.csv")

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')

# Add x, y for lat long
ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')

class_options = list(ferm_locations_final.Class.unique())

ferm_locations_final.Acquisition_date = pd.to_datetime(ferm_locations_final.Acquisition_date)
ferm_locations_final.Extract_date = pd.to_datetime(ferm_locations_final.Extract_date)
ferm_locations_final.Arrival_date = pd.to_datetime(ferm_locations_final.Arrival_date)

# Bokeh (Faster)

In [4]:
date_range_slider = pn.widgets.DatetimeRangePicker(
    name='Show Samples Between:',
    enable_time = False,
    start=dt.datetime(2021, 12, 1), end=dt.datetime(2022, 8, 1),
    value=(dt.datetime(2021, 12, 1), dt.datetime(2022, 8, 1))
)

In [5]:
country_choice = pn.widgets.MultiChoice(name='Country', value=['France', 'Thailand'],
    options=list(ferm_locations_final.Country_obtained.unique()))

In [6]:
class_choice = pn.widgets.MultiChoice(name='Class', value=['Dairy', 'Fish'],
    options=list(ferm_locations_final.Class.unique()))

In [7]:
p = bokeh.plotting.figure(
        frame_height=400,
        frame_width=700,
        x_axis_label="Latitude",
        y_axis_label="Longitude",
        tools =['wheel_zoom', 'pan', 'reset', 'tap'],
        x_axis_type = 'mercator',
        y_axis_type = 'mercator',
        x_range = [ferm_locations_final.x.min()-100000, ferm_locations_final.x.max() + 100000],
        y_range = [ferm_locations_final.y.min()-100000, ferm_locations_final.y.max() + 100000]

    )

Class_list=ferm_locations_final.Class.unique()
Class_len = len(Class_list)

# Mapping of color for glyphs
mapper = bokeh.transform.factor_cmap('Class', palette=bokeh.palettes.Category20[15], factors=Class_list)

p.toolbar_location = 'above'
p.add_tile(tile_provider)
p.add_tools(bokeh.models.BoxZoomTool(match_aspect=True))

hover = bokeh.models.HoverTool()

hover.tooltips = """
<font face="Arial" size="3">
<div>
    <div><strong>Ferment Class:  </strong>@Class</div>
    <div><strong>Name: </strong>@x</div>
</div>
</font>
"""
p.add_tools(hover)

url = "https://github.com/lianamerk/hovercal/blob/main/examples/fruit_hovercal_1.png?raw=true"
taptool = p.select(type=bokeh.models.TapTool)
taptool.callback = bokeh.models.OpenURL(url=url)

In [8]:
chosen_cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ["#1f77b3"] * len(ferm_locations_final),
    }
)

In [9]:
# point available
cds = bokeh.models.ColumnDataSource(ferm_locations_final)

In [10]:
circle = p.circle(
    source=chosen_cds, x="x", y="y", fill_color="color", line_color="color",
)

In [14]:
# JavaScript code for the callback stored as a string
jscode = """
let selected_class = class_choice.value;

let point_visible = source_visible.data;
let point_available = source_available.data;

point_visible.x = []
point_visible.y = []
point_visible.color = []

for (let i = 0; i < point_available.x.length; i++) {
    if (selected_class.includes(point_available.Class[i])) {
        point_visible.x.push(point_available.x[i]);
        point_visible.y.push(point_available.y[i]);
        point_visible.color.push('#1f77b3');

    }
}
console.log(point_visible.x.length);
console.log(point_visible.y.length);
console.log(point_visible.color.length);

source_visible.change.emit(); 
"""

In [16]:
class_choice.jscallback(value=jscode, args=dict(source_available=cds, source_visible=chosen_cds, class_choice=class_choice))


Callback(args={'source_available': ColumnDataSource(id='1056', ...), 'source_visible': ColumnDataSource(id='1055', ...), 'class_choice': MultiChoice(name='Class', options=['Dairy', 'Control', ...], value=['Dairy', 'Fish'])}, code={'value': "\nlet selected_class = class_choice.value;\n\nlet point_visible = source_visible.data;\nlet point_available = source_available.data;\n\npoint_visible.x = []\npoint_visible.y = []\npoint_visible.color = []\n\nfor (let i = 0; i < point_available.x.length; i++) {\n    if (selected_class.includes(point_available.Class[i])) {\n        point_visible.x.push(point_available.x[i]);\n        point_visible.y.push(point_available.y[i]);\n        point_visible.color.push('#1f77b3');\n\n    }\n}\nconsole.log(point_visible.x.length);\nconsole.log(point_visible.y.length);\nconsole.log(point_visible.color.length);\n\nsource_visible.change.emit(); \n"}, name='Callback00105')

In [17]:
p_pane = pn.pane.Bokeh(p)

In [18]:
map_panel = pn.Row(p, pn.Spacer(width=15), pn.Column(class_choice))
map_panel

Row
    [0] Bokeh(Figure)
    [1] Spacer(width=15)
    [2] Column
        [0] MultiChoice(name='Class', options=['Dairy', 'Control', ...], value=['Dairy', 'Fish'])